# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.2/78.2 GB disk)


In [ ]:
!unzip -q ../train_data.zip -d ../

In [ ]:
!unzip -q ../DD.zip -d ../

# 1. Predict

`segment/predict.py` runs YOLOv5 instance segmentation inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/predict`. Example inference sources are:

```shell
python segment/predict.py --source 0  # webcam
                             img.jpg  # image
                             vid.mp4  # video
                             screen  # screenshot
                             path/  # directory
                             'path/*.jpg'  # glob
                             'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                             'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files.download('/content/drive/MyDrive/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import imageio
import os

def create_gif(image_folder, gif_name):
    images = []
    filenames = sorted(os.listdir(image_folder))
    for filename in filenames:
        image_path = os.path.join(image_folder, filename)
        images.append(imageio.imread(image_path))
    imageio.mimsave(gif_name, images, fps=1)

image_folder = '/content/DD'
gif_name = '/content/animated.gif'
create_gif(image_folder, gif_name)

In [ ]:
#!python segment/predict.py --weights yolov5s-seg.pt --img 640 --conf 0.25 --source data/images

#!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.10 --source ../animated.gif
!python detect.py --weights /content/drive/MyDrive/best.pt --img 640 --conf 0.10 --source ../20220508.00.Indian.IRImage.png
#display.Image(filename='runs/predict-seg/exp/zidane.jpg', width=600)

detect: weights=['/content/drive/MyDrive/best.pt'], source=../20220508.00.Indian.IRImage.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
image 1/1 /content/20220508.00.Indian.IRImage.png: 416x640 1 CS, 67.8ms
Speed: 0.5ms pre-process, 67.8ms inference, 102.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp24


# 2. Validate
Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag.

In [ ]:
# Download COCO val
!bash data/scripts/get_coco.sh --val --segments  # download (780M - 5000 images)

######################################################################## 100.0%
######################################################################## 100.0%


In [ ]:
# Validate YOLOv5s-seg on COCO val
!python segment/val.py --weights yolov5s-seg.pt --data coco.yaml --img 640 --half

segment/val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s-seg.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val-seg, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-2-gc9d47ae Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s-seg summary: 224 layers, 7611485 parameters, 0 gradients, 26.4 GFLOPs
val: Scanning /content/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:03<00:00, 1361.31it/s]
val: New cache created: /content/datasets/coco/val2017.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% 157/157 [01:54<00:00,  1.37it/s]
                   all       5000      36335      0.673      0.517      0.566      0.373

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'TensorBoard' #@param ['TensorBoard', 'Comet', 'ClearML']

if logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train-seg
elif logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  import clearml; clearml.browser_login()

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 2 --epochs 70 --data custom_data.yaml --weights yolov5x.pt --nosave --cache

train: weights=yolov5x.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=70, batch_size=2, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 4.73 KiB | 2.37 MiB/s, done.
From https://github.com/ultralytics/yolov5
   cec1b9b..a270b4f  master     -> origin/master
github: ⚠️ YOLOv5 is out of date by 3 commits. Use `git pull`

# Appendix

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s-seg')  # yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.